In [ ]:
import cmipld
from cmipld.utils.ldparse import *



In [ ]:
cd ../

/Users/daniel.ellis/WIPwork/CMIP7-CVs


In [66]:
import glob,os,json
files = [f.split(os.sep)[-2] for f in glob.glob('*/')]
files

['project', 'experiment']

In [135]:
f = 'experiment'

import subprocess
import yaml

result = subprocess.run(
    ["git", "show", "main:.github/ISSUE_TEMPLATE/experiment.yml"],
    capture_output=True,
    text=True,
    check=True  # Raises exception on error
)

# Parse YAML
dyaml = yaml.safe_load(result.stdout)

ids = set([item['id'] for item in dyaml['body'] if 'id' in item])
ids


{'activity',
 'alias',
 'branch_date',
 'description',
 'issue_category',
 'issue_kind',
 'minimum_number_of_years',
 'model_realms',
 'parent_experiment',
 'parent_experiment_other',
 'references',
 'required_model_realms',
 'start_date',
 'tier',
 'ui_label',
 'validation_key'}

In [136]:
dropdown = []
multi=[]
for entry in dyaml['body']:
    if entry['type'] == 'dropdown':
        dropdown.append(entry['id'])
        if entry['attributes'].get('multiple',False) :
            multi.append(entry['id'])
            

files = glob.glob(f'{f}/*.json')

def extract(val):
    if isinstance(val, list):
        return [extract(v) for v in val]
    if isinstance(val, dict):
        val = next(iter(val.values()))
        print(val.get('validation_key'))
        return val.get('validation_key', val.get('@id'))
    return val


In [ ]:
from urllib.parse import urlencode
from typing import OrderedDict
urls = []

for entry in files:
    print(f"Processing file: {entry}")
    # jd = json.load(open(f))
    # open the file
    jd = cmipld.get(entry,depth=3)
    

    
    
    match = OrderedDict()
    
    match['template'] = f'{f}.yml'
    match['title'] = f"Modify: {f.title()}: {jd.get('validation_key',jd['@id'])}"
    match['issue_kind'] = '"Modify"'
    
    for key in ids:
        if key in jd:
            value = jd.get(key)
            # print(f"Found matching id: {key}")
            entry = extract(value)
            
            if key in multi:
                entry = ','.join([f'"{e}"' for e in list(entry)])
            elif key in dropdown:
                entry = f'"{entry}"'
            elif isinstance(entry, list): 
                # not sure when this will happen, but just in case
                entry = ', '.join(entry)
            
            match[key] = entry

    query_string = 'https://github.com/WCRP-CMIP/CMIP7-CVs/issues/new?' + urlencode(match)
    print(query_string)
    
    urls.append(query_string)



Processing file: experiment
[Cache HIT] experiment/g7-1p5k-sai.json (depth=3)
Found matching id: minimum_number_of_years
Found matching id: references
Found matching id: alias
Found matching id: description
Found matching id: parent_experiment
Found matching id: ui_label
Found matching id: validation_key
Found matching id: activity
GeoMIP
Found matching id: start_date
Found matching id: tier
https://github.com/WCRP-CMIP/CMIP7-CVs/issues/new?template=experiment.yml&title=Modify%3A+Experiment%3A+G7-1p5K-SAI&issue_kind=%22Modify%22&minimum_number_of_years=50&references=&alias=&description=Stratospheric+aerosol+injection+to+reduce+warming+to+1.5K+%0A+Stratospheric+aerosol+injection+to+reduce+warming+to+1.5K+above+pre-industrial+levels&parent_experiment=%22%5B%27none%27%5D%22&ui_label=G7-1.5K-SAI&validation_key=G7-1p5K-SAI&activity=%22GeoMIP%22&start_date=none&tier=%220%22
Processing file: experiment
[Cache HIT] experiment/amip-p4k.json (depth=3)
Found matching id: minimum_number_of_years
F